### Setup paths

In [1]:
import sys
import os

In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

### Imports

In [3]:
from data_pull_and_prep.audio_from_yt import download_audio

### Step 0: (Optional) Download audio files from YouTube

I'm using podcasts from youtube in this project as an example. This step is not needed if you already have an mp3 file, skip to step 1.

In [5]:
# Usage
video_url = "https://www.youtube.com/watch?v=vcEVgN4eET8"  # Replace with your video URL
video_name = "test3"  # Replace with your video name
output_dir = project_root+"/data/testing/audio_0/"  # Replace with your output directory

download_audio(video_url, video_name, output_dir)

Audio downloaded: /Users/rishikeshdhayarkar/rag-audio-indexing/data/testing/audio_0/test3.mp3


### Step 1: Convert mp3 file to text and generate time stamps for each character

In [3]:
import data_pull_and_prep.utils as utils
import data_pull_and_prep.data_preparation as data_prep
import textwrap

Convert audio to text using Open AI whisper

In [5]:
audio_file_path = project_root+"/data/testing/audio_0/test3.mp3"
transcription = data_prep.transcribe(audio_file_path)

/Users/rishikeshdhayarkar/rag-audio-indexing/rag-audio-env/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = t

Transcribed output contains an id, piece of converted text, start time and end time in the audio clip for this text. 

In [6]:
print(len(transcription))
print(f"id: {transcription[5][0]}")
print(f"text: {transcription[5][1]}")
print(f"start time: {transcription[5][2]}")
print(f"end time: {transcription[5][3]}")

34
id: 5
text:  what he called new dirt, like they're new dirt, like to make it super like with safety features
start time: 26.76
end time: 31.44


For each such segment(above cell), calculate the time stamp for each character in text by interpolation.

But why do we need character level time stamps?
Character level timestamps provide the flexibility to create textchunks of any size.

In [15]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

In [16]:
# transcription_with_char_timestamps = data_prep.map_characters_to_timestamps(transcription)

In [17]:
print(f"Total number of characters: {len(transcription_with_char_timestamps)}")
transcription_with_char_timestamps[:5]

Total number of characters: 157283


[(' ', 0.0),
 ('T', 0.06449438202247192),
 ('h', 0.12898876404494383),
 ('e', 0.19348314606741573),
 (' ', 0.25797752808988766)]

Save character level timestamps

In [18]:
# utils.save_as_pickle_file(directory=project_root+"/data/testing/audio_0/",
#                     filename="transcription_with_char_timestamps.pkl",
#                     data=transcription_with_char_timestamps)

Create custom chunks using SentenceSplitter from Llamaindex.

In [19]:
custom_chunking_obj = data_prep.CreateCustomTextChunks(transcription_with_char_timestamps)
text_chunks_with_timestamps = custom_chunking_obj.create_custom_text_chunks()

In [20]:
print(f"Number of text chunks: {len(text_chunks_with_timestamps)}")

Number of text chunks: 42


In [22]:
print(textwrap.fill(str(text_chunks_with_timestamps[-1]), width=80))

("have like how precious the earth is um I hope that it I hope that it has that
effect uh because I you know I think there's a big component to to
interplanetary travel that kind of taps into this kind of manifest destiny and
clonation like the human desire to conquer territory and expand um the footprint
of civilization that sometimes feels much more rooted in like dominance and
conquest than curiosity wonder and um and obviously I think there's you know
maybe an existential imperative for at some point or a strategic or and security
one but um but I hope that what feels inevitable at this moment I mean you know
Elon Musk and what he's doing with SpaceX and Jeff Bezos and others it feels
like it's not an if it's uh it's a when at this point I hope it also underscores
like the need to protect what we have here yeah and it's I hope is the curiosity
that drives that exploration and I hope the exploration will give us a deeper
appreciation of the thing we have back home and that earth wil